In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 

In [2]:
# Make spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [6]:
# Deliverable 2 Q1
# Filter Data and create new DataFrame to retrieve all rows where total_votes count is equal to 20 or greater.
vine_df_q1 = vine_df.filter("total_votes>=20")

In [12]:
# Deliverable 2 Q2
# Retrieve all rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_df_q2 = vine_df_q1.filter(vine_df_q1["helpful_votes"]/vine_df_q1["total_votes"] >= 0.5)

In [27]:
# Deliverable 2 Q3 & Q4
# Retrieve all rows where vine == 'Y' and where vine == 'N'
vine_paid_df = vine_df_q2.filter(vine_df_q2["vine"]=="Y")
vine_unpaid_df = vine_df_q2.filter(vine_df_q2["vine"]=="N")

In [24]:
# Deliverable 2 Q5
# Determine total number of reviews, number of 5 star reviews, and percentage of 5 star reviews per paid vs unpaid

# Vine Program
count_paid = vine_paid_df.count()
five_star_paid = vine_paid_df.filter("star_rating==5").count()
percent_five_star_paid = (five_star_paid / count_paid) * 100
print(f'''
The total number of reviews for the vine program is: {count_paid}
The total number of 5 star reviews for the vine program is: {five_star_paid}
The percentage of 5 star reviews for the vine program is: %{percent_five_star_paid:.2f}
''')


The total number of reviews for the vine program is: 107
The total number of 5 star reviews for the vine program is: 56
The percentage of 5 star reviews for the vine program is: %52.34



In [26]:
# Not in the vine program
count_unpaid = vine_unpaid_df.count()
five_star_unpaid = vine_unpaid_df.filter("star_rating==5").count()
percent_five_star_unpaid = (five_star_unpaid / count_unpaid) * 100
print(f'''
The total number of reviews not in the vine program is: {count_unpaid}
The total number of 5 star reviews not in the vine program is: {five_star_unpaid}
The percentage of 5 star reviews not in the vine program is: %{percent_five_star_unpaid:.2f}
''')



The total number of reviews not in the vine program is: 39869
The total number of 5 star reviews not in the vine program is: 21005
The percentage of 5 star reviews not in the vine program is: %52.69

